# Download do Dataset


In [3]:
!gdown https://drive.google.com/uc?id=1gnQKgBru-Em2WNMSJDtBfSnqSOYhcvNN&export=download

Downloading...
From: https://drive.google.com/uc?id=1gnQKgBru-Em2WNMSJDtBfSnqSOYhcvNN
To: /content/mask_dataset.rar
277MB [00:03, 73.5MB/s]


In [ ]:
!unrar x -r mask_dataset.rar ./mask_dataset/

# Instalação Wandb

In [ ]:
%%capture
!pip install wandb -q

In [ ]:
!wandb login

In [6]:
import wandb

#Importa bibliotecas


In [7]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model  import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.svm     import SVC
from sklearn.tree    import DecisionTreeClassifier

from imutils import paths
import numpy as np
import progressbar
import h5py
import random
import pickle
import os

# HDF5

In [8]:
class HDF5DatasetWriter:
  def __init__(self, dims, outputPath, dataKey="images",bufSize=1000):

    if os.path.exists(outputPath):
      raise ValueError("O caminho para arquivo não foi encontrado ou não pode ser sobrescrito.", outputPath)

    self.db = h5py.File(outputPath, "w")
    self.data = self.db.create_dataset(dataKey, dims,dtype="float")#compression='gzip')
    self.labels = self.db.create_dataset("labels", (dims[0],),dtype="int")

    self.bufSize = bufSize
    self.buffer = {"data": [], "labels": []}
    self.idx = 0

  def add(self, rows, labels):

    self.buffer["data"].extend(rows)
    self.buffer["labels"].extend(labels)

    if len(self.buffer["data"]) >= self.bufSize:
      self.flush()

  def flush(self):

    i = self.idx + len(self.buffer["data"])
    self.data[self.idx:i] = self.buffer["data"]
    self.labels[self.idx:i] = self.buffer["labels"]
    self.idx = i
    self.buffer = {"data": [], "labels": []}

  def storeClassLabels(self, classLabels):
    
    dt = h5py.special_dtype(vlen=str) 
    labelSet = self.db.create_dataset("label_names",(len(classLabels),), dtype=dt)
    labelSet[:] = classLabels

  def close(self):
    
    if len(self.buffer["data"]) > 0:
      self.flush()

    self.db.close()

# Extração de característica 

In [9]:
def feature_extraction(dataset,output,buffer_size,bs):

		print("[INFO] loading images...")
		imagePaths = list(paths.list_images(dataset))
		random.shuffle(imagePaths)

		labels = [p.split(os.path.sep)[-2] for p in imagePaths]
		le = LabelEncoder()
		labels = le.fit_transform(labels)

		# load the VGG16 network
		print("[INFO] loading network...")
		model = VGG16(weights="imagenet", include_top=False)

		dataset = HDF5DatasetWriter((len(imagePaths), 512 * 7 * 7),	output, dataKey="features", bufSize=buffer_size)
		dataset.storeClassLabels(le.classes_)

		# initialize the progress bar
		widgets = ["Extracting Features: ", progressbar.Percentage(), " ", progressbar.Bar(), " ", progressbar.ETA()]
		pbar = progressbar.ProgressBar(maxval=len(imagePaths),widgets=widgets).start()

		# loop over the images in batches
		for i in np.arange(0, len(imagePaths), bs):
    
			batchPaths = imagePaths[i:i + bs]
			batchLabels = labels[i:i + bs]
			batchImages = []

			# loop over the images and labels in the current batch
			for (j, imagePath) in enumerate(batchPaths):
     
				image = load_img(imagePath, target_size=(224, 224))
				image = img_to_array(image)
    
				image = np.expand_dims(image, axis=0)
				image = imagenet_utils.preprocess_input(image)

				batchImages.append(image)

			batchImages = np.vstack(batchImages)
			features = model.predict(batchImages, batch_size=bs)

			features = features.reshape((features.shape[0], 512 * 7 * 7))

			dataset.add(features, batchLabels)
			pbar.update(i)

		dataset.close()
		pbar.finish()

# Variáveis

In [10]:
!mkdir ./hdf5/

In [11]:
dataset = "mask_dataset"

output  = "./hdf5/features.hdf5"

buffer_size = 1000
bs = 32

In [12]:
feature_extraction(dataset,output,buffer_size,bs)

[INFO] loading images...
[INFO] loading network...
58900480/58889256 [==============================] - 0s 0us/step


Extracting Features: 100% |####################################| Time:  0:07:07


In [13]:
db = h5py.File(output,mode='r')
list(db.keys())

['features', 'label_names', 'labels']

In [14]:
[str(i,'utf-8') for i in db["label_names"]]

['Mask', 'NoMask']

## Regressão Logística 

In [21]:
def train_RL():

  wandb.init(project="Transfer_Learning_Feature_Extraction", entity="imd1114")
  
  db = h5py.File(output,mode='r')
  print("Database keys {0:}".format(list(db.keys())))

  i = int(db["labels"].shape[0] * 0.75)  

  X_train = db["features"][:i]
  y_train = db["labels"][:i]
  X_test  = db["features"][i:]
  y_test  = db["labels"][i:]
  
  configs = {
    'C': 0.01
  }  
  
  config = wandb.config
  
  model = LogisticRegression(solver="lbfgs", multi_class="auto", C=wandb.config.C, n_jobs=-1)  
  
  # Treino
  model.fit(X_train, y_train)

  print("[INFO] Avaliando...")
  wandb.log({"Treino":    model.score(X_train, y_train), 
             "Validação": model.score(X_test, y_test)})

  db.close()

In [22]:
# grid search WandB
print("[INFO] tuning hyperparameters...")
sweep_config = {
  'method': 'grid',
  'parameters': {
      'C': {
          'values': [0.01, 0.1, 1.0, 10.0]
      }
  }
}

sweep_id = wandb.sweep(sweep_config)

[INFO] tuning hyperparameters...
Create sweep with ID: vxfo6b1i
Sweep URL: https://wandb.ai/3w3rt0n/uncategorized/sweeps/vxfo6b1i


In [23]:
wandb.agent(sweep_id, function=train_RL)

wandb: Agent Starting Run: wkfk8wjc with config:
wandb: 	C: 0.01
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: WARNING Ignored wandb.init() arg entity when running a sweep


Database keys ['features', 'label_names', 'labels']
[INFO] Avaliando...


Treino,1.0
Validação,0.99565
_runtime,26
_timestamp,1631576814
_step,0


Treino,▁
Validação,▁
_runtime,▁
_timestamp,▁
_step,▁


wandb: Agent Starting Run: l7qo34uy with config:
wandb: 	C: 0.1
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: WARNING Ignored wandb.init() arg entity when running a sweep


...

wandb: Agent Starting Run: 22p1nmv4 with config:
wandb: 	C: 10
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: WARNING Ignored wandb.init() arg entity when running a sweep


Database keys ['features', 'label_names', 'labels']
[INFO] Avaliando...


Treino,1.0
Validação,0.99565
_runtime,15
_timestamp,1631576885
_step,0


Treino,▁
Validação,▁
_runtime,▁
_timestamp,▁
_step,▁


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


## SVM

In [24]:
def train_SVC():
  
  wandb.init(project="Transfer_Learning_Feature_Extraction_SVC", entity="imd1114")
  
  db = h5py.File(output,mode='r')
  print("Database keys {0:}".format(list(db.keys())))

  i = int(db["labels"].shape[0] * 0.75)  

  X_train = db["features"][:i]
  y_train = db["labels"][:i]
  X_test  = db["features"][i:]
  y_test  = db["labels"][i:]

  sc = StandardScaler()
  sc.fit(X_train)
  
  X_train_std = sc.transform(X_train)
  X_test_std = sc.transform(X_test)
    
  configs = {
    'gamma': 0.5,
    'C': 1.8,
  }
  
  config = wandb.config
  
  model = SVC(kernel='rbf', random_state=0, gamma=wandb.config.gamma, C=wandb.config.C)
  model.fit(X_train_std, y_train)
  
  model.fit(X_train, y_train)

  print("[INFO] Avaliação...")  
  wandb.log({"Treino": model.score(X_train_std, y_train), 
             "Validação": model.score(X_test_std, y_test)})

  def plot_data():
      from matplotlib.colors import ListedColormap
      import matplotlib.pyplot as plt

      markers = ('s', 'x')
      colors = ('red', 'blue')
      cmap = ListedColormap(colors[:len(np.unique(y_test))])
      for idx, cl in enumerate(np.unique(y)):
          plt.scatter(x=X[y == cl, 0], y=X[y == cl, 1],
                c=cmap(idx), marker=markers[idx], label=cl)

      wandb.log({"Data": plt})

  db.close()

In [25]:
# grid search WandB
print("[INFO] tuning hyperparameters...")
sweep_config = {
  'method': 'grid',
  'parameters': {
      'gamma': {
          'values': [0.1, 0.3, 0.5, 0.8]
      },
      'C': {
          'values': [0.5, 0.8, 1.0, 1.5]
      }
  }
}

sweep_id = wandb.sweep(sweep_config)

[INFO] tuning hyperparameters...
Create sweep with ID: ebumb8sj
Sweep URL: https://wandb.ai/3w3rt0n/uncategorized/sweeps/ebumb8sj


In [ ]:
wandb.agent(sweep_id, function=train_SVC)

wandb: Agent Starting Run: gttxwzdf with config:
wandb: 	C: 0.5
wandb: 	gamma: 0.1
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: WARNING Ignored wandb.init() arg entity when running a sweep


Database keys ['features', 'label_names', 'labels']
[INFO] Avaliação...


Treino,0.52256
Validação,0.4913
_runtime,74
_timestamp,1631577045
_step,0


Treino,▁
Validação,▁
_runtime,▁
_timestamp,▁
_step,▁


wandb: Agent Starting Run: puh14myy with config:
wandb: 	C: 0.5
wandb: 	gamma: 0.3
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: WARNING Ignored wandb.init() arg entity when running a sweep


...

Database keys ['features', 'label_names', 'labels']
[INFO] Avaliação...


Treino,0.52256
Validação,0.4913
_runtime,74
_timestamp,1631577764
_step,0


Treino,▁
Validação,▁
_runtime,▁
_timestamp,▁
_step,▁


wandb: Agent Starting Run: ra80hw0w with config:
wandb: 	C: 1
wandb: 	gamma: 0.5
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: WARNING Ignored wandb.init() arg entity when running a sweep


Database keys ['features', 'label_names', 'labels']


## Árvore de decissão

In [ ]:
def train_DT():
  
  wandb.init(project="Transfer_Learning_Feature_Extraction_SVC", entity="imd1114")
  
  db = h5py.File(output,mode='r')
  print("Database keys {0:}".format(list(db.keys())))

  i = int(db["labels"].shape[0] * 0.75)  

  X_train = db["features"][:i]
  y_train = db["labels"][:i]
  X_test  = db["features"][i:]
  y_test  = db["labels"][i:]
  
  configs = {
    'max_depth': 1,
    'min_samples_leaf': 1
  }
  
  config = wandb.config
  
  model = DecisionTreeClassifier(max_depth=wandb.config.max_depth, min_samples_leaf=wandb.config.min_samples_leaf, random_state=0)
    
  # Treino
  model.fit(X_train, y_train)
  
  print("[INFO] Avaliação...")  
  wandb.log({"Treino": model.score(X_train, y_train), 
             "Validação": model.score(X_test, y_test)})

  def plot_data():
      from matplotlib.colors import ListedColormap
      import matplotlib.pyplot as plt

      markers = ('s', 'x')
      colors = ('red', 'blue')
      cmap = ListedColormap(colors[:len(np.unique(y_test))])
      for idx, cl in enumerate(np.unique(y)):
          plt.scatter(x=X[y == cl, 0], y=X[y == cl, 1],
                c=cmap(idx), marker=markers[idx], label=cl)

      wandb.log({"Data": plt})

  db.close()

In [ ]:
# grid search WandB
print("[INFO] tuning hyperparameters...")
sweep_config = {
  'method': 'grid',
  'parameters': {
      'max_depth': {
          'values': [1, 3, 5, 7, 10]
      },
      'min_samples_leaf': {
          'values': [1, 3, 5, 8, 10]
      }
  }
}

sweep_id = wandb.sweep(sweep_config)

[INFO] tuning hyperparameters...
Create sweep with ID: 3khiyckb
Sweep URL: https://wandb.ai/3w3rt0n/uncategorized/sweeps/3khiyckb


In [ ]:
wandb.agent(sweep_id, function=train_DT)

wandb: Agent Starting Run: dcek6tp7 with config:
wandb: 	max_depth: 1
wandb: 	min_samples_leaf: 1
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: WARNING Ignored wandb.init() arg entity when running a sweep


Database keys ['features', 'label_names', 'labels']
[INFO] evaluating...


Train Accuracy,0.96507
Test Accuracy,0.96087
_runtime,6
_timestamp,1631542222
_step,0


Train Accuracy,▁
Test Accuracy,▁
_runtime,▁
_timestamp,▁
_step,▁


wandb: Agent Starting Run: ydvwc28k with config:
wandb: 	max_depth: 1
wandb: 	min_samples_leaf: 3
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: WARNING Ignored wandb.init() arg entity when running a sweep


...

wandb: Agent Starting Run: 33udtewd with config:
wandb: 	max_depth: 10
wandb: 	min_samples_leaf: 10
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: WARNING Ignored wandb.init() arg entity when running a sweep


Database keys ['features', 'label_names', 'labels']
[INFO] evaluating...


Train Accuracy,0.97962
Test Accuracy,0.96087
_runtime,9
_timestamp,1631542607
_step,0


Train Accuracy,▁
Test Accuracy,▁
_runtime,▁
_timestamp,▁
_step,▁


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
